# Setup

In [ ]:
import re
import sys
from pathlib import Path
from glob import glob
from typing import Union, List
from subprocess import check_output, check_call

import ray, ray.tune
import yaml, json

GRAPHGYM_ROOT = Path("~/Documents").expanduser() / "GraphGym"
main_py = GRAPHGYM_ROOT / "run" / "main.py"
configs_gen_py = GRAPHGYM_ROOT / "run" / "configs_gen.py"
agg_batch_py = GRAPHGYM_ROOT / "run" / "agg_batch.py"
assert main_py.exists()
assert configs_gen_py.exists()
assert agg_batch_py.exists()

In [ ]:
# install custom contrib files to the graphgym folder
check_output([sys.executable, str(Path("") / "install_contrib_files.py"), str(GRAPHGYM_ROOT)]);
print("Installed customized GraphGym module with contrib files from the `contrib` folder.")

In [ ]:
def call_python(path: Union[str, Path], args: List[str], verbose: bool = False) -> str:
    """Call a Python subprocess and return its stdout output as a string."""
    if verbose:
        check_call([sys.executable, str(path), *args])
        output = None
    else:
        output = check_output([sys.executable, str(path), *args]).decode("utf8")
    return output


def run_config(config_path: Union[str, Path], repeats: int = 1, verbose: bool = False) -> str:
    return call_python(
        main_py, ["--cfg", str(config_path), "--repeat", str(repeats)], verbose=verbose
    )


def gen_configs(
    config_path: Union[str, Path], grid_path: Union[str, Path], sample_num: int = -1
) -> str:
    args = ["--config", str(config_path), "--grid", str(grid_path)]
    if sample_num > 0:
        args = args + ["--sample", "--sample_num", str(sample_num)]
    return call_python(configs_gen_py, args)

# A simple grid experiment

Let's first generate configuration files using `configs_gen.py` script.

In [ ]:
classic_config = Path("").absolute() / "configs" / "classic.yaml"
classic_grid = Path("").absolute() / "configs" / "classic_grid.txt"
assert classic_config.exists()
assert classic_grid.exists()
print(gen_configs(classic_config, classic_grid))
config_dir = Path("").absolute() / "configs" / f"{classic_config.stem}_grid_{classic_grid.stem}"
assert config_dir.exists()
config_paths = list(glob(str(config_dir / "*.yaml")))
assert len(config_paths) > 0

Now, let's create a ray function that runs a config.

We need to be a little careful to about working with Ray since Ray can run on a distributed cluster.

1. pass the configuration file as a Python dictionary
2. write that dictionary into a new configuration file
3. run the experiment using `main.py` from GraphGym
4. read the results of a trial from `stats.json` files in `train`, `val`, `test` result folders

In [ ]:
def read_graphgym_stats_file(path):
    path = Path(path)
    return [json.loads(line) for line in (path).read_text().strip().split("\n") if len(line) > 0]


def experiment_fn(config):
    # our custom config from the ray config
    config = config["custom_config"]
    repeats = config.get("repeats", 1)
    trial_name = config["trial_name"]
    grid_name = config["grid_name"]

    # write the config file passed by dict (since we want to allow distributed ray clusters)
    config_file_contents = config["config_file_contents"]
    config_file_contents.setdefault("dataset", dict())
    dataset_path = Path("/tmp/graphgym_datasets")
    dataset_path.mkdir(parents=True, exist_ok=True)
    config_file_contents["dataset"]["dir"] = str(dataset_path)
    config_path = Path("configs") / grid_name / f"{trial_name}.yaml"
    config_path.parent.mkdir(parents=True, exist_ok=True)
    config_path.write_text(yaml.dump(config_file_contents))

    # run the actual experiment
    run_config(config_path, repeats, verbose=config.get("verbose", True))

    # find the results
    result_path = (
        Path(re.sub(r"(^|/)configs/", r"\1results/", str(config_path.parent))) / trial_name
    )
    all_stats = dict()
    for key in ["train", "val", "test"]:
        stats_file = result_path / "agg" / key / "stats.json"
        print(f"Looking under {stats_file} for stats file")
        if stats_file.exists():
            all_stats[key] = read_graphgym_stats_file(stats_file)

    # finally, we'll return the final result
    return dict(all_stats)

Here, we convert the configuration files to a ray config, which is a dictionary.

In [ ]:
# we will convert config_paths into a list of configs with config read out using yaml
configs = [
    {
        "repeats": 1,
        "config_file_contents": yaml.safe_load(Path(config_path).read_text()),
        "trial_name": Path(config_path).stem,
        "grid_name": Path(config_path).parent.stem,
        "verbose": False,
    }
    for config_path in config_paths
]

# we'll only specify one field, custom_config because we're generating samples ourselves
ray_configs = {"custom_config": ray.tune.grid_search(configs)}

# by specifying resources, we're implicitly specifying how many jobs should run in parallel
resources_per_trial = {"cpu": 1, "gpu": 0.5}

# now, we define experiments using our ray function
experiments = ray.tune.Experiment(
    "graphgym_experiment",
    experiment_fn,
    config=ray_configs,
    resources_per_trial=resources_per_trial,
)

Let's test the experiment_fn locally first to see if it runs properly.

In [ ]:
# we should test our ray function locally first
result = experiment_fn({"custom_config": dict(configs[0], verbose=True)})

Run the experiment grid on the cluster

In [ ]:
results = ray.tune.run_experiments(experiments=experiments)

Analyze the results

In [ ]:
for experiment in results:
    config = experiment.config
    print(config)
    print(f"Validation accuracy was {experiment.last_result['val'][-1]['accuracy']}")
    print("#######################")